In [1]:
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'
trip_data_file_path = './Data/yellow_tripdata_2021-06.parquet'


In [2]:
trips = pq.read_table(trip_data_file_path)
trips = trips.to_pandas()
trips.dropna(inplace = True)

# data cleaning
trips = trips.query('passenger_count > 0.0 and trip_distance > 0.0 and trip_distance < 100 and total_amount > 0.0 and RatecodeID < 99')
trips = trips.query('tpep_pickup_datetime >= "2021-06-01" and tpep_pickup_datetime <= "2021-06-30 23:59:59"')
trips['store_and_fwd_flag'].loc[trips['store_and_fwd_flag'] == 'N'] = 0
trips['store_and_fwd_flag'].loc[trips['store_and_fwd_flag'] == 'Y'] = 1

manhattan_zones = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 104, 105, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]
trips = trips.loc[trips["PULocationID"].isin(manhattan_zones)]
trips = trips.loc[trips["DOLocationID"].isin(manhattan_zones)]


In [3]:
# add tip percentage and buckets
trips["tip_pct"] = trips["tip_amount"] / trips["total_amount"]
trips["tip_bin"] = pd.cut(trips.tip_pct, 
                       bins = [0.0, 0.10, 0.2, 0.5, 1.0], 
                       labels = [0, 1, 2, 3], #0:"[0%, 10%]", 1:"(10%, 20%]", 2:"(20%, 50%]", 3:"(50%, 100%]"
                       include_lowest = True)


In [4]:
# convert dates to integers for passing to PyTorch
trips["PU_day_in_june_2021"] = abs(pd.to_datetime(trips["tpep_pickup_datetime"]) - pd.to_datetime('2021-06-01')).dt.days + 1
trips["PU_time_hour"] = pd.to_datetime(trips["tpep_pickup_datetime"]).dt.hour
trips["PU_time_min"] = pd.to_datetime(trips["tpep_pickup_datetime"]).dt.minute

trips["DO_day_in_june_2021"] = abs(pd.to_datetime(trips["tpep_dropoff_datetime"]) - pd.to_datetime('2021-06-01')).dt.days + 1
trips["DO_time_hour"] = pd.to_datetime(trips["tpep_dropoff_datetime"]).dt.hour
trips["DO_time_min"] = pd.to_datetime(trips["tpep_dropoff_datetime"]).dt.minute


In [5]:
# add weather data
trips['Date'] = pd.to_datetime(trips['tpep_pickup_datetime']).dt.date

weather = pd.read_csv("https://raw.githubusercontent.com/weiangela/STAT453---Taxi-Prediction/main/Data/2021NYCWeather.csv", 
                      parse_dates=[0],
                      date_format='%Y-%m-%d',
                      usecols = [0, 2, 5, 8, 11, 14, 16], 
                      dtype = {'avg_temp':'float64', 
                               'avg_dew_pt':'float64', 
                               'avg_humidity':'float64',
                               'avg_wind_speed':'float64',
                               'avg_pressure':'float64',
                               'total_precipitation':'float64'})

weather['Date'] = pd.to_datetime(weather['Date'], format='%m/%d/%y').dt.date

trips = trips.join(weather.set_index('Date'), on = "Date")
trips.drop(["Date", "tpep_pickup_datetime", "tpep_dropoff_datetime"], axis = 1, inplace = True)

In [6]:
# convert VendorID, PULocationID, DOLocationID, RatecodeID, store_and_fwd_flag, payment_type, and tip_bin columns into onehot encoding for tensor model maniuplation
# all other columns will be considered to be values and not representative variables

pd.get_dummies(trips, columns=['VendorID', 'PULocationID', 'DOLocationID', 'RatecodeID', 'store_and_fwd_flag', 'payment_type', 'tip_bin'], dtype = int)


,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,...,store_and_fwd_flag_0,store_and_fwd_flag_1,payment_type_1,payment_type_2,payment_type_3,payment_type_4,tip_bin_0,tip_bin_1,tip_bin_2,tip_bin_3
0,1.0,0.90,5.0,3.0,0.5,2.20,0.0,0.3,11.00,2.5,...,1,0,1,0,0,0,0,1,0,0
8,1.0,5.25,18.5,0.5,0.5,4.46,0.0,0.3,26.76,2.5,...,1,0,1,0,0,0,0,1,0,0
9,1.0,1.74,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,...,1,0,0,1,0,0,1,0,0,0
11,3.0,4.30,14.0,3.0,0.5,3.55,0.0,0.3,21.35,2.5,...,1,0,1,0,0,0,0,1,0,0
15,1.0,0.90,5.0,3.0,0.5,0.00,0.0,0.3,8.80,2.5,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710721,1.0,2.34,9.5,0.5,0.5,3.33,0.0,0.3,16.63,2.5,...,1,0,1,0,0,0,0,0,1,0
2710722,1.0,2.36,9.5,0.5,0.5,3.33,0.0,0.3,16.63,2.5,...,1,0,1,0,0,0,0,0,1,0
2710723,1.0,1.16,7.0,0.5,0.5,2.16,0.0,0.3,12.96,2.5,...,1,0,1,0,0,0,0,1,0,0
2710724,1.0,1.50,6.5,0.5,0.5,2.06,0.0,0.3,12.36,2.5,...,1,0,1,0,0,0,0,1,0,0


In [7]:
# write to parquet file
trips.to_parquet('./Data/cleaned_taxi_data.parquet')

In [9]:
# how to open parquet file
import torch
import numpy as np
temp = pq.read_table('./Data/cleaned_taxi_data.parquet')
temp = temp.to_pandas()

torch_temp = torch.tensor(temp.values)
